In [10]:
import tensorflow, keras
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
import csv
import cv2

In [11]:
def get_answer_list(file_name, screen_num):
    axis_list = list()
    with open(file_name, 'r') as csv_file:
        reader = csv.DictReader(csv_file, delimiter='\t')
        for row in reader:
            if int(row['screen_num']) == screen_num:
                coordinate = [int(row['x1']), int(row['y1']),\
                              int(row['x2']), int(row['y2'])]
                axis_list.append(coordinate)
    return axis_list

In [12]:
#이미지 비율 조정
def rescale_img(img, scale_percent):
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    reimg = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return reimg

In [13]:
# 이미지 데이터 읽기
def show_img(img):
    img_resized=rescale_img(img, 30)
    cv2.imshow('mser', img_resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
test_img_url = '/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/00-screenshot/kr.co.company.hwahae/3.png'
answer_url = '/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/01-csv/clickable/kr.co.company.hwahae.csv'
answer_list = get_answer_list(answer_url, 3)
img = cv2.imread(test_img_url)
print(len(answer_list))

38


In [18]:
# Shi-Tomasi Corner Detection
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
corners = cv2.goodFeaturesToTrack(gray,1000,0.01,10)
corners = np.int0(corners)
for i in corners:
    x,y = i.ravel()
    cv2.circle(img,(x,y),3,(255,0,0),10)

In [9]:
# Canny Detection
def canny_detection(img):
    axis_dict = dict()
    gray_vis = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=8.0, tileGridSize=(16,16))
    cl1 = clahe.apply(gray_vis)
    edges = cv2.Canny(cl1, 120, 125)
    #print("[EDGES] ", edges.shape)
    arr = np.array(edges)
    result = np.where(arr==np.amax(arr))
    # print("[RESULT]", result)
    result_arr = list(zip(result[0], result[1]))
    #print(result_arr)
    for index, axis in result_arr:
        if index not in axis_dict.keys():
            axis_dict.update({index:list()})
        axis_dict[index].append(axis)
    return axis_dict

result = canny_detection(img)
for key in result.keys():
    for element in result[key]:
        cv2.circle(img, (element, key), \
                         1, (255,0,0), 3)  
show_img(img)

In [ ]:
# Harris Corner
def harris_corner(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # find Harris corners
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,2,3,0.04)
    dst = cv2.dilate(dst,None)
    ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)

    # Now draw them
    res = np.hstack((centroids,corners))
    res = np.int0(res)
    print(res)
    img[res[:,1],res[:,0]]=[0,0,255]
    img[res[:,3],res[:,2]] = [0,255,0]
    return res
img = harris_corner(img)

In [14]:
#MSER Algorithm
def mser_algo(img):
    mser = cv2.MSER_create()
    gray_vis = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #clahe = cv2.createCLAHE(clipLimit=8.0, tileGridSize=(16,16))
    #cl1 = clahe.apply(clahe)
    regions, _ = mser.detectRegions(gray_vis)
    hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
    return hulls
#hulls = mser_algo(img)

#for i, contour in enumerate(hulls):
#    x, y, w, h = cv2.boundingRect(contour)
#    cv2.rectangle(img, (x, y+h), (x+w, y), (255, 0, 0), 3)
#cv2.polylines(img, hulls, 1, (255, 0, 0))

In [49]:
#contouring
gray_vis = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#ret,thresh = cv2.threshold(gray_vis,127,255,0)
clahe = cv2.createCLAHE(clipLimit=8.0, tileGridSize=(16,16))
cl1 = clahe.apply(gray_vis)
edges = cv2.Canny(cl1, 120, 125)
arr = np.array(edges)
result = np.where(arr==np.amax(arr))

contours, hierarchy = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
#img = cv2.drawContours(img, contours, -1, (0, 255, 0), 3)


cnt = contours[0]
for cnt in contours:
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)
    rect_area = w*h
    extend = float(area) / rect_area
    cv2.rectangle(img, (x, y), (x+w+int(extend), y+h+int(extend)), (255, 0, 0), 3)

print(np.array(contours))

[array([[[ 995, 2922]],

       [[ 994, 2923]],

       [[ 990, 2923]],

       [[ 987, 2926]],

       [[ 987, 2933]],

       [[ 990, 2936]],

       [[1254, 2936]],

       [[1257, 2933]],

       [[1257, 2926]],

       [[1254, 2923]],

       [[1250, 2923]],

       [[1249, 2922]],

       [[1239, 2922]],

       [[1238, 2923]],

       [[1237, 2923]],

       [[1236, 2922]],

       [[1082, 2922]],

       [[1081, 2923]],

       [[1080, 2923]],

       [[1079, 2922]]], dtype=int32)
 array([[[ 994, 2923]],

       [[ 995, 2922]],

       [[1079, 2922]],

       [[1080, 2923]],

       [[1081, 2923]],

       [[1082, 2922]],

       [[1236, 2922]],

       [[1237, 2923]],

       [[1238, 2923]],

       [[1239, 2922]],

       [[1249, 2922]],

       [[1250, 2923]],

       [[1253, 2923]],

       [[1257, 2927]],

       [[1257, 2932]],

       [[1253, 2936]],

       [[ 991, 2936]],

       [[ 987, 2932]],

       [[ 987, 2927]],

       [[ 991, 2923]]], dtype=int32)
 array([[[ 5

In [7]:
# 정답 데이터 그리기
answer_list
for axis in answer_list:
    cv2.rectangle(img, (axis[0], axis[1]), (axis[2], axis[3]), (0, 0, 255), 3)

In [14]:
#get Orientation and magnitude
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(img.shape)
X = cv2.Sobel(gray, cv2.CV_32F, 1, 0)
Y = cv2.Sobel(gray, cv2.CV_32F, 0, 1)

orien = cv2.phase(X, Y, angleInDegrees=True)
mag = cv2.magnitude(X, Y)


#for i, contour in enumerate(hulls):
#x, y, w, h = cv2.boundingRect(contour)
#    cv2.rectangle(img, (x, y+h), (x+w, y), (255, 0, 0), 3)
show_img(mag)



(2960, 1440, 3)


In [46]:
nonzero_orien = np.nonzero(orien)
nonzero_orien_loc = zip(nonzero_orien[0], nonzero_orien[1])
#for x,y in nonzero_orien_loc:
#    print(x, y)

In [48]:
nonzero_mag = np.nonzero(mag)
nonzero_mag_loc = zip(nonzero_mag[0], nonzero_mag[1])
for x,y in nonzero_mag_loc:
    print(x, y)

0 1
0 203
0 205
0 408
0 410
0 613
0 615
0 818
0 820
0 1023
0 1024
0 1025
0 1228
0 1230
0 1232
0 1233
0 1438
1 0
1 1
1 2
1 202
1 203
1 204
1 205
1 206
1 407
1 408
1 409
1 410
1 411
1 612
1 613
1 614
1 615
1 616
1 817
1 818
1 819
1 820
1 821
1 1022
1 1023
1 1024
1 1025
1 1026
1 1227
1 1228
1 1229
1 1230
1 1231
1 1232
1 1233
1 1234
1 1437
1 1438
1 1439
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
2 33
2 34
2 35
2 36
2 37
2 38
2 39
2 40
2 41
2 42
2 43
2 44
2 45
2 46
2 47
2 48
2 49
2 50
2 51
2 52
2 53
2 54
2 55
2 56
2 57
2 58
2 59
2 60
2 61
2 62
2 63
2 64
2 65
2 66
2 67
2 68
2 69
2 70
2 71
2 72
2 73
2 74
2 75
2 76
2 77
2 78
2 79
2 80
2 81
2 82
2 83
2 84
2 85
2 86
2 87
2 88
2 89
2 90
2 91
2 92
2 93
2 94
2 95
2 96
2 97
2 98
2 99
2 100
2 101
2 102
2 103
2 104
2 105
2 106
2 107
2 108
2 109
2 110
2 111
2 112
2 113
2 114
2 115
2 116
2 117
2 118
2 119
2 120
2 121
2 122
2 123
2 124
2 125
2 126


9 1144
9 1145
9 1146
9 1147
9 1227
9 1228
9 1230
9 1232
9 1233
9 1234
9 1235
9 1236
9 1237
9 1238
9 1239
9 1240
9 1241
9 1242
9 1243
9 1244
9 1245
9 1246
9 1247
9 1252
9 1253
9 1254
9 1255
9 1256
9 1257
9 1258
9 1259
9 1327
9 1328
9 1329
9 1330
9 1331
9 1332
9 1333
9 1334
9 1335
9 1336
9 1337
9 1338
9 1339
9 1358
9 1359
9 1360
9 1361
9 1362
9 1363
9 1364
9 1365
9 1366
9 1367
9 1368
9 1369
9 1370
9 1378
9 1379
9 1380
9 1381
9 1382
9 1383
9 1384
9 1385
9 1386
9 1387
9 1388
9 1389
9 1390
9 1391
9 1437
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
10 13
10 14
10 15
10 16
10 17
10 44
10 45
10 46
10 47
10 48
10 49
10 50
10 51
10 52
10 53
10 54
10 55
10 56
10 57
10 58
10 84
10 85
10 86
10 87
10 88
10 89
10 90
10 91
10 106
10 107
10 108
10 109
10 110
10 111
10 112
10 113
10 114
10 115
10 202
10 203
10 205
10 206
10 220
10 221
10 222
10 223
10 224
10 225
10 226
10 227
10 228
10 229
10 230
10 231
10 232
10 233
10 234
10 235
10 236
10 237
10 238
10 239
10 240
10 241
10 242
10 243

12 442
12 443
12 444
12 445
12 446
12 447
12 448
12 449
12 450
12 451
12 452
12 453
12 454
12 455
12 456
12 503
12 504
12 505
12 506
12 507
12 508
12 509
12 510
12 511
12 512
12 520
12 521
12 522
12 523
12 524
12 525
12 526
12 527
12 528
12 529
12 530
12 531
12 532
12 533
12 534
12 535
12 536
12 537
12 551
12 552
12 553
12 554
12 555
12 556
12 557
12 558
12 559
12 560
12 561
12 562
12 563
12 564
12 565
12 566
12 567
12 568
12 569
12 612
12 613
12 615
12 617
12 619
12 620
12 621
12 622
12 623
12 624
12 625
12 626
12 627
12 628
12 629
12 630
12 631
12 632
12 633
12 634
12 635
12 637
12 638
12 684
12 685
12 686
12 687
12 688
12 689
12 690
12 691
12 692
12 693
12 694
12 695
12 696
12 697
12 698
12 699
12 700
12 714
12 715
12 716
12 717
12 718
12 719
12 720
12 721
12 722
12 723
12 724
12 725
12 726
12 727
12 728
12 729
12 730
12 731
12 735
12 736
12 737
12 738
12 739
12 740
12 741
12 742
12 743
12 744
12 745
12 746
12 747
12 748
12 749
12 750
12 751
12 752
12 753
12 758
12 759
12 760
12 761

16 1236
16 1237
16 1238
16 1245
16 1246
16 1247
16 1248
16 1249
16 1304
16 1305
16 1306
16 1307
16 1308
16 1309
16 1310
16 1311
16 1323
16 1324
16 1325
16 1326
16 1327
16 1328
16 1329
16 1330
16 1335
16 1336
16 1337
16 1338
16 1339
16 1340
16 1341
16 1342
16 1354
16 1355
16 1356
16 1357
16 1358
16 1359
16 1360
16 1361
16 1366
16 1367
16 1368
16 1369
16 1370
16 1371
16 1372
16 1373
16 1376
16 1377
16 1378
16 1379
16 1380
16 1389
16 1390
16 1391
16 1392
16 1393
16 1394
16 1395
16 1437
17 1
17 3
17 4
17 5
17 6
17 7
17 8
17 14
17 15
17 16
17 17
17 18
17 19
17 20
17 22
17 23
17 24
17 25
17 26
17 27
17 28
17 29
17 41
17 42
17 43
17 44
17 45
17 46
17 47
17 48
17 54
17 55
17 56
17 57
17 58
17 59
17 60
17 61
17 81
17 82
17 83
17 84
17 85
17 86
17 87
17 88
17 107
17 108
17 109
17 110
17 111
17 112
17 113
17 114
17 115
17 202
17 203
17 205
17 206
17 211
17 212
17 213
17 214
17 215
17 216
17 217
17 218
17 219
17 220
17 222
17 223
17 224
17 225
17 226
17 227
17 228
17 229
17 230
17 232
17 233
17 23

18 1255
18 1256
18 1257
18 1258
18 1262
18 1263
18 1264
18 1265
18 1266
18 1267
18 1268
18 1269
18 1270
18 1271
18 1272
18 1273
18 1274
18 1275
18 1276
18 1277
18 1278
18 1279
18 1280
18 1286
18 1287
18 1288
18 1289
18 1290
18 1291
18 1292
18 1293
18 1294
18 1295
18 1296
18 1297
18 1298
18 1299
18 1304
18 1305
18 1306
18 1309
18 1310
18 1311
18 1323
18 1324
18 1325
18 1327
18 1328
18 1329
18 1336
18 1337
18 1338
18 1339
18 1340
18 1341
18 1342
18 1343
18 1354
18 1355
18 1356
18 1358
18 1359
18 1360
18 1367
18 1368
18 1369
18 1370
18 1371
18 1372
18 1373
18 1374
18 1376
18 1377
18 1378
18 1379
18 1389
18 1390
18 1391
18 1392
18 1393
18 1394
18 1395
18 1437
19 1
19 3
19 4
19 5
19 6
19 7
19 8
19 14
19 15
19 16
19 18
19 19
19 20
19 22
19 23
19 24
19 27
19 28
19 29
19 41
19 42
19 43
19 45
19 46
19 47
19 55
19 56
19 57
19 59
19 60
19 61
19 80
19 81
19 82
19 83
19 84
19 85
19 86
19 87
19 107
19 108
19 109
19 110
19 111
19 112
19 113
19 114
19 115
19 202
19 203
19 205
19 206
19 209
19 210
19 2

20 1136
20 1137
20 1145
20 1146
20 1147
20 1149
20 1150
20 1151
20 1227
20 1228
20 1230
20 1232
20 1234
20 1235
20 1236
20 1237
20 1238
20 1239
20 1240
20 1241
20 1242
20 1243
20 1247
20 1248
20 1249
20 1250
20 1253
20 1254
20 1255
20 1256
20 1257
20 1258
20 1262
20 1263
20 1264
20 1265
20 1273
20 1274
20 1275
20 1276
20 1277
20 1278
20 1279
20 1280
20 1284
20 1285
20 1286
20 1287
20 1288
20 1289
20 1290
20 1295
20 1296
20 1297
20 1298
20 1299
20 1300
20 1301
20 1304
20 1305
20 1306
20 1307
20 1308
20 1309
20 1310
20 1311
20 1322
20 1323
20 1324
20 1325
20 1327
20 1328
20 1329
20 1337
20 1338
20 1339
20 1341
20 1342
20 1343
20 1354
20 1355
20 1356
20 1358
20 1359
20 1360
20 1368
20 1369
20 1370
20 1372
20 1373
20 1374
20 1376
20 1377
20 1378
20 1379
20 1388
20 1389
20 1390
20 1391
20 1392
20 1393
20 1394
20 1437
21 1
21 3
21 4
21 5
21 6
21 7
21 8
21 13
21 14
21 15
21 16
21 17
21 18
21 19
21 20
21 22
21 23
21 24
21 25
21 26
21 27
21 28
21 29
21 41
21 42
21 43
21 45
21 46
21 47
21 55
21 

22 1109
22 1110
22 1111
22 1112
22 1113
22 1114
22 1131
22 1132
22 1133
22 1135
22 1136
22 1137
22 1145
22 1146
22 1147
22 1149
22 1150
22 1151
22 1227
22 1228
22 1230
22 1232
22 1234
22 1235
22 1236
22 1237
22 1238
22 1239
22 1240
22 1241
22 1242
22 1243
22 1244
22 1245
22 1246
22 1247
22 1248
22 1249
22 1250
22 1253
22 1254
22 1255
22 1256
22 1257
22 1258
22 1261
22 1262
22 1263
22 1264
22 1265
22 1266
22 1267
22 1268
22 1269
22 1270
22 1271
22 1272
22 1273
22 1274
22 1275
22 1276
22 1277
22 1278
22 1279
22 1280
22 1283
22 1284
22 1285
22 1286
22 1287
22 1288
22 1289
22 1290
22 1291
22 1292
22 1293
22 1294
22 1295
22 1296
22 1297
22 1298
22 1299
22 1300
22 1301
22 1302
22 1304
22 1305
22 1306
22 1307
22 1308
22 1309
22 1310
22 1311
22 1317
22 1318
22 1319
22 1320
22 1321
22 1322
22 1323
22 1325
22 1327
22 1328
22 1329
22 1330
22 1331
22 1332
22 1333
22 1334
22 1337
22 1338
22 1339
22 1341
22 1342
22 1343
22 1354
22 1355
22 1356
22 1358
22 1359
22 1360
22 1368
22 1369
22 1370
22 1372


24 927
24 928
24 929
24 930
24 931
24 932
24 933
24 934
24 935
24 936
24 937
24 939
24 940
24 941
24 942
24 943
24 944
24 945
24 946
24 947
24 948
24 949
24 950
24 951
24 952
24 953
24 954
24 956
24 957
24 958
24 959
24 961
24 962
24 963
24 964
24 965
24 966
24 967
24 968
24 969
24 971
24 972
24 973
24 974
24 975
24 976
24 977
24 978
24 979
24 1023
24 1024
24 1025
24 1026
24 1027
24 1028
24 1029
24 1030
24 1031
24 1032
24 1033
24 1034
24 1035
24 1036
24 1037
24 1038
24 1039
24 1040
24 1041
24 1042
24 1043
24 1044
24 1045
24 1046
24 1047
24 1048
24 1049
24 1050
24 1051
24 1052
24 1053
24 1054
24 1055
24 1056
24 1057
24 1058
24 1059
24 1060
24 1061
24 1062
24 1063
24 1064
24 1065
24 1066
24 1067
24 1068
24 1069
24 1070
24 1071
24 1072
24 1073
24 1074
24 1075
24 1076
24 1077
24 1078
24 1079
24 1080
24 1081
24 1082
24 1083
24 1084
24 1085
24 1086
24 1087
24 1088
24 1089
24 1090
24 1091
24 1092
24 1093
24 1094
24 1095
24 1096
24 1097
24 1098
24 1099
24 1100
24 1101
24 1102
24 1103
24 1104
2

KeyboardInterrupt: 

In [9]:
import pandas as pd
df = pd.DataFrame(data = mag)
df.head(50)

,0,1,2,3,4,5,6,7,8,9,...,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439
0,0.0,74.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,74.0,52.325901,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,32.526913,216.009262,420.326050,512.0,512.000000,512.000000,512.000000,512.0,507.024658,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Edge Detection을 통해서 버튼 요소들 모두 잘라내기
'''
1. 이미지 읽어들이기 샘플 이미지: abysrium pole
2. Canny Detection
3. 좌표리스트 저장
4. 좌표리스트 읽어 이미지 crop리스트 저장
'''
test_img_url = '/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/00-screenshot/kr.co.company.hwahae/3.png'
img = cv2.imread(test_img_url)
print(img.shape)

result = canny_detection(img)
#cv2.rectangle(img, (result[0][0], result[0][1]), \
#                   (result[0][2], result[0][3]), \
#                   (255,0,0), 10)  

#for key in result.keys():
#    if len(result[key]) < 4:
#        print(key, len(result[key]))

    
#if cv2.waitKey(1) & 0XFF == ord('q'):
#    cv2.destroyAllWindows()
#print(result)
for key in result.keys():
    for element in result[key]:
        cv2.circle(img, (element, key), \
                         1, (255,0,0), 3)  
print(result.keys())
#plt.imshow(img)


In [ ]:
test_img_url = '/home/kimsoohyun/Downloads/Screenshot_20200418-220702_Abyssrium Pole.jpg'

img = cv2.imread(test_img_url)
img_gray = cv2.imread(test_img_url, cv2.COLOR_BGR2GRAY)

mser = cv2.MSER_create()
regions, _ = mser.detectRegions(img_gray)

img_clone = img.copy()
for r in regions:
    (x,y,w,h) = cv2.boundingRect(np.reshape(r,(-1,1,2)))
    cv2.rectangle(img_clone,(x,y),(x+w,y+h),(0,0,255),1)
    
print(img.shape)
#plt.imshow(img_clone)
cv2.imshow('mser', img_clone)
cv2.waitKey(0)
cv2.destroyAllWindows()

!ls dataset/03-models/model
#모델 export
model = tensorflow.keras.models.load_model('./dataset/03-models/model/my_model.h5')

In [ ]:
# CNN predict

In [ ]:
# 클릭요소를 찾아서 이미지에 표현하기